# Lab 6: Visualizing the training process of neural networks. Hyperparameter tuning.

In this lab, you will learn how to use [wandb](https://wandb.ai/) to visualize the training process of neural networks. We are going to build and train a feed-forward neural network for recognizing handwritten digits of the MNIST dataset. The training process will be visualized in the wandb dashboard, which will allow us to monitor the loss and accuracy of the model in real-time.

---



Feel free to create an account at [wandb.ai](https://wandb.ai/) before starting this lab.

### A simple example of how to use wandb in a typical training loop is shown below:

```python
import wandb

wandb.login() # Log in to your wandb account

# Start a new run

some_config = {
    'learning_rate': 0.01,
    'layer_1_size': 128,
    'layer_2_size': 64,
    'batch_size': 32
} # This is just an example of a configuration dictionary, you can put anything you want here

wandb.init(project='mnist-classifier', config=some_config) # start a new run and log parameters

# Here you would prepare your data, and initialize the model, optimizer, etc.

# Training loop
for epoch in range(100):
    ...
    wandb.log({'loss': loss, 'accuracy': accuracy})
    # This will send the loss and accuracy to wandb and you can visualize it in the dashboard

# End of the run
wandb.finish()
```

The most important part is the `wandb.log()` function, which sends the data to the wandb dashboard. You can log any metric you want, not just loss and accuracy. The value passed to the function must be a dictionary.


## Exercise 1: Prepare data for training a mnist classifier (2 points)

Before you start training a neural network, you need to prepare the data. In this exercise, you will prepare the MNIST dataset of handwritten digits for training a classifier. You should:

1. Load the MNIST dataset using from `data/mnist_train.csv` and `data/mnist_test.csv` files.
2. Normalize the data to the range [0, 1].
3. Encode the labels using one-hot encoding.
4. Create a PyTorch `Dataset` object for the training and test sets.
5. Create a PyTorch `DataLoader` object for the training and test sets.

In [1]:
import pandas as pd

# Load the data
train_data = pd.read_csv('data/mnist-train.csv')
test_data = pd.read_csv('data/mnist-test.csv')

# Extract and OHE the labels
train_labels = pd.get_dummies(train_data['label'], columns=['label'])
test_labels = pd.get_dummies(test_data['label'], columns=['label'])

# Normalize the data
train_data = train_data.drop(columns='label') / 255
test_data = test_data.drop(columns='label') / 255

import torch

# create tensors
train_data_tensor = torch.tensor(train_data.values, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels.values, dtype=torch.float32)
test_data_tensor = torch.tensor(test_data.values, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels.values, dtype=torch.float32)

# create datasets
train_dataset = torch.utils.data.TensorDataset(train_data_tensor, train_labels_tensor)
test_dataset = torch.utils.data.TensorDataset(test_data_tensor, test_labels_tensor)

# create dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


## Exercise 2: Prepare the architecture of the neural network (2 points)

In this exercise, you will prepare the architecture of the neural network. You should:

1. Create a neural network class that inherits from `torch.nn.Module`.
2. The neural network should have at least one hidden layer.
3. Use ReLU activation functions after each but the output layer.
4. Use a softmax activation function in the output layer to get the probabilities of each class.

**Feel free to experiment with the architecture of your network** - try adding more hidden layers, changing the number of neurons in each layer, etc. You can also add a dropout layer or some other regularization technique and see if it improves the performance of your model.

In [2]:
# Your code goes here
...
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self,hidden_size=128):
        super(NeuralNetwork,self).__init__()
        self.fc1=nn.Linear(784,hidden_size)
        self.fc2=nn.Linear(hidden_size,10)
        self.relu=nn.ReLU()
        self.softmax=nn.Softmax()
        
    def forward(self,x):
        y=self.fc1(x)
        x=self.relu(y)
        output=self.fc2(x)
        return output
        

## *Training PyTorch models on GPU

**GPUs are optimized for performing matrix operations in parallel.** Although we call them "graphics processing units", they are actually very powerful processors that can be used for any kind of parallel computation, including training deep neural networks. In fact, data science is one of the most common applications of GPUs today, as can be seen by the revenue of companies like NVIDIA over the past few years. NVIDIA is a monopolist in the GPU market - in 2023, the company owned 92% of the data center GPU market share. As for 31 July, the 2024 revenue of NVIDIA was 60.92 billion USD, while the total revenue of 2020 was $10.92 billion. If someone benefits from the current deep learning hype, it is certainly NVIDIA.

If you happen to have an NVIDIA GPU in your computer, you can use it to train your deep learning models, as PyTorch has excellent support for CUDA, which is NVIDIA's parallel computing API. To train a model on GPU, you need to explicitly tell PyTorch to move the model and the data to the GPU. 

Here is an example training loop that uses the GPU:

```python
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # Check if a GPU is available

# Initialize the model and move it to the GPU
model = SomeNeuralNetwork().to(device)   # Move the model to the GPU

for epoch in range(100):
    for batch in data_loader:
        X, y = batch
        X, y = X.to(device), y.to(device)   # Move the tensors to GPU
        
        y_pred = model(X)   # Perform a forward pass (on the GPU)
        loss = criterion(y_pred, y)   # Compute the loss (still on the GPU)
        
        ...  # The rest of the training loop
        
        y_pred = y_pred.detach().cpu()   # Move the predictions back to the CPU to do anything else with them
```

Note that **the model and all the tensors it uses for computation should be moved to the GPU**. You can do this by calling the `.to(device)` method on the model and the data tensors. If you want to move the data back to the CPU (to process it further, calculate metrics, visualize), you call the `.cpu()` method on the tensor.

**Doing calculations on the GPU, you should be wary of few things:**

* **The GPU has a limited amount of memory**, so you should be careful not to run out of memory. A typical graphics card has a few gigabytes of memory, so you should be fine with most models and datasets. However, moving very large tensors to the GPU can cause out-of-memory errors. That's one of the reasons why we use a dataloader and process the data in batches.
* While the GPU is much faster than the CPU for large matrix operations, **transferring data between the CPU and the GPU is slow**. Therefore, it is best to minimize the number of data transfers between the CPU and the GPU.

## Exercise 3: Prepare the training loop (2 points)

In this exercise, you will prepare the training loop. You should:

1. Initialize the neural network.
2. Define the loss function.
3. Define the optimizer.
4. Pass a dictionary with the configuration to wandb. This dictionary should contain all the hyperparameters of our model, including the learning rate, the size of the hidden layers, batch size, etc.
4. Train the neural network. Each epoch should consist of a training and validation phase. You should log the loss and accuracy of the training and validation sets using wandb.
5. Open you project at [wandb.ai](https://wandb.ai/) and see how cool it is!

### Saving and loading the model
As training can take some time, it is a good idea to save the model's state dictionary (its learned weights) to a file after training. You can do this with the following code:

    torch.save(vae.state_dict(), 'vae.pth')
    
To load the model from the file, you can use the following code:

    vae.load_state_dict(torch.load('vae.pth'))

In [4]:
# Your code goes here
...
import wandb
def train(model, train_loader, val_loader, epochs=100):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # lr to learing rate
    loss = nn.MSELoss() 
    
    train_loss_history = []
    val_loss_history = []

    wandb.init(project='mnist-classifier')
    
    for epoch in range(epochs): 
       
        model.train()   
        train_loss = 0  
        
        for X_batch, y_batch in train_loader:   # load data batch-by-batch
                #petla dla loadera             
            optimizer.zero_grad()   
         
            y_pred = model(X_batch)  
            
            batch_loss = loss(y_pred, y_batch) 
       
            batch_loss.backward()   #
          
            optimizer.step()    
            
            train_loss += batch_loss.item()
        
        train_loss = train_loss / len(train_loader)
        train_loss_history.append(train_loss)   
        print(f'Epoch: {epoch}')
        print(f'Train loss: {train_loss}')
                  
        model.eval()   
        val_loss = 0   
        
        for X_batch, y_batch in val_loader:
            
            y_pred = model(X_batch)
            val_loss += loss(y_pred, y_batch).item()  
            
        val_loss = val_loss / len(val_loader)
        val_loss_history.append(val_loss)
        print(f'Validation loss: {val_loss}')
        wandb.log({'train_loss': train_loss,'val_loss': val_loss})

    wandb.finish()
    return model, train_loss_history, val_loss_history

In [21]:
model = NeuralNetwork()
# hiiden_size 
model, train_metrics, val_metrics = train(model, train_loader, test_loader)

Epoch: 0
Train loss: 0.01718532071063916
Validation loss: 0.011567301596678532
Epoch: 1
Train loss: 0.01057759656123817
Validation loss: 0.01046400202569156
Epoch: 2
Train loss: 0.009190680351232489
Validation loss: 0.009147459703583877
Epoch: 3
Train loss: 0.008411475603034098
Validation loss: 0.008917407628101186
Epoch: 4
Train loss: 0.007899963137507439
Validation loss: 0.009067964249144727
Epoch: 5
Train loss: 0.007525463636716207
Validation loss: 0.00853318425294119
Epoch: 6
Train loss: 0.0071572855754444996
Validation loss: 0.008431129790202807
Epoch: 7
Train loss: 0.006894038282086452
Validation loss: 0.008188411545085783
Epoch: 8
Train loss: 0.006656435206035773
Validation loss: 0.008370360504603757
Epoch: 9
Train loss: 0.006472232178474466
Validation loss: 0.008222013122042099
Epoch: 10
Train loss: 0.006285937825838725
Validation loss: 0.008310456111946831
Epoch: 11
Train loss: 0.006145105107376972
Validation loss: 0.008130602733479046
Epoch: 12
Train loss: 0.00599927624681343

train_loss,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▂▃▃▃▃▃▃▃▃▄▄▄▄▅▄▄▄▅▅
train_loss,0.00339
val_loss,0.00934


## Exercise 4: Easy hyperparameter tuning with wandb (2 points)

Wandb allows you to perform hyperparameter tuning by automatically creating multiple runs with different hyperparameters and logging the performance of each run. Below is a brief instruction to `wandb` hyperparameter tuning, but you are more than welcome to find more information in the [official wandb guide](https://docs.wandb.ai/guides/sweeps/).

Your task is to use wandb to perform hyperparameter tuning of the neural network, trying different values of the learning rate, batch size, and the size of the hidden layers. You can use the following hyperparameters:

First, we need to define a dictionary with the hyperparameters that we want to tune. For example:

```python
parameters = {
    'learning_rate': {'values': [0.01, 0.001, 0.0001]},
    'batch_size': {'values': [32, 64, 128]},
    'layer_1_size': {'values': [64, 128, 256]},
    'layer_2_size': {'values': [32, 64, 128]}
}
```

Then we need to create a dictionary with the configuration of the run:

```python
sweep_config = {
    'name': 'mnist-sweep',
    'method': 'grid',   # grid search, you can also try 'random' or 'bayes'
    'metric': {'goal': 'minimize', 'name': 'val_loss'},
    'parameters': parameters,   # that's the dictionary with the hyperparameters
}
```

Finally, we can use the `wandb.sweep` function to perform hyperparameter tuning:

```python
sweep_id = wandb.sweep(sweep_config, project='mnist-classifier')
```

After that, we can finally run the sweep:

```python
wandb.agent(sweep_id, function=train)
```
where `train` is a function that trains the model and logs the metrics to wandb. This function should take a `config` argument, which will contain the hyperparameters of the run. That is how wandb knows which hyperparameters to tune.

1. Rewrite the VAE training loop into a function that takes a single dictionary `parameters` as an argument, initializes the model, optimizer, and criterion, and trains the model for a fixed number of epochs. The function should log the loss and accuracy of the training and validation sets to wandb.
2. Create a dictionary with the hyperparameters that you want to tune.
3. Create a sweep configuration dictionary.
4. Run the sweep and monitor the results in the wandb dashboard.

In [3]:
import wandb
wandb.login(key='d8df1d7205f98685d35ca86003d2c2f1e06c4215')
wandb.init(project='mnist_classifier')
def train(model, train_loader, val_loader, epochs=100):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # lr to learing rate
    loss = nn.MSELoss() #funkcja kosztu
    
    train_loss_history = []
    val_loss_history = []
    
    for epoch in range(epochs): # epoch to przejscie przez caly dataset
        # jak dataloader wyciagnie wszystko juz z datasetu to minie wtedy jedna epoka
        model.train()   # set the model to training mode (some layers may behave differently in training and evaluation)
        train_loss = 0  # this variable will accumulate the training loss
        
        for X_batch, y_batch in train_loader:   # load data batch-by-batch
                #petla dla loadera             
            optimizer.zero_grad()   # clear the gradients
            # trzeba wyzerowac gradienty na optimizerze
            y_pred = model(X_batch)  # forward pass
            
            batch_loss = loss(y_pred, y_batch) # compute the loss
            # liczymy blad (ustalony MSE)
            batch_loss.backward()   # compute the gradients
            # metoda backward odpala algorytm do liczenia gradientu (wiec nie musimy go liczyc recznie)
            optimizer.step()    # update the weights
            # "hej, masz policzone gradienty, dobierz wagi tak, aby loss byl mniejszy nastepnym razem."
            train_loss += batch_loss.item() # accumulate training loss
        
        train_loss = train_loss / len(train_loader) # compute the average loss
        train_loss_history.append(train_loss)   # save the loss for plotting
        ###
        print(f'Epoch: {epoch}')
        print(f'Train loss: {train_loss}')
                  
        model.eval()   
        val_loss = 0   
        
        for X_batch, y_batch in val_loader:
            
            y_pred = model(X_batch)
            val_loss += loss(y_pred, y_batch).item()  
            
        val_loss = val_loss / len(val_loader)
        val_loss_history.append(val_loss)
        print(f'Validation loss: {val_loss}')
        wandb.log({'train_loss': train_loss,'val_loss': val_loss})
        #####

    wandb.finish()
    return model, train_loss_history, val_loss_history
    # your code goes here
    ...

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: orzol-robert (orzol-robert-uj). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\HP\_netrc


Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "C:\Users\HP\anaconda34\envs\pum\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\HP\anaconda34\envs\pum\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 325, in check_internal_messages
    self._loop_check_status(
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 235, in _loop_check_status
    local_handle = request()
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\interface\interface.py", line 914, in deliver_internal_messages
    return self._deliver_internal_messages(internal_message)
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\interface\inter

In [4]:

parameters = {
    'learning_rate': {'values': [0.01, 0.001, 0.0001]},
    'batch_size': {'values': [32, 64, 128]},
    'layer_1_size': {'values': [64, 128, 256]},
    'layer_2_size': {'values': [32, 64, 128]}
}

sweep_config = {
    'name': 'mnist-sweep',
    'method': 'bayes',
    'metric': {'goal': 'maximize', 'name': 'accuracy'}, # if we want to maximize the accuracy
    # remember to log the metric that you want to maximize or minimize!
    'parameters': parameters,
}
model = NeuralNetwork()
sweep_id = wandb.sweep(sweep_config, project='mnist-classifier')    # This will create a new sweep
wandb.agent(sweep_id, lambda:train(model, train_loader, test_loader))   # This will start the hyperparameter tuning process

Create sweep with ID: aioy2jvb
Sweep URL: https://wandb.ai/orzol-robert-uj/mnist-classifier/sweeps/aioy2jvb


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: wu2ccjmp with config:
wandb: 	batch_size: 64
wandb: 	layer_1_size: 256
wandb: 	layer_2_size: 64
wandb: 	learning_rate: 0.01


Epoch: 0
Train loss: 0.01727109913478295
Validation loss: 0.01147494759386816


Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_40212\3272628806.py", line 17, in <lambda>
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_40212\2011325668.py", line 47, in train
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 451, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 403, in wrapper_fn
    return func(self, *args, **kwargs)
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 393, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 1922, in log
    self._log(data=data, step=step, commit=commit)
  File "C:\Users\HP\anaconda34\envs\pum\li

Epoch: 0
Train loss: 0.010581786843761801
Validation loss: 0.009611591509284494


...

Exception in thread Thread-16:
Traceback (most recent call last):
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_40212\3272628806.py", line 17, in <lambda>
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_40212\2011325668.py", line 47, in train
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 451, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 403, in wrapper_fn
    return func(self, *args, **kwargs)
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 393, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 1922, in log
    self._log(data=data, step=step, commit=commit)
  File "C:\Users\HP\anaconda34\envs\pum\li

Epoch: 0
Train loss: 0.007103644604360064
Validation loss: 0.00830484672855681


Exception in thread Thread-17:
Traceback (most recent call last):
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_40212\3272628806.py", line 17, in <lambda>
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_40212\2011325668.py", line 47, in train
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 451, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 403, in wrapper_fn
    return func(self, *args, **kwargs)
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 393, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\HP\anaconda34\envs\pum\lib\site-packages\wandb\sdk\wandb_run.py", line 1922, in log
    self._log(data=data, step=step, commit=commit)
  File "C:\Users\HP\anaconda34\envs\pum\li

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x00000135AD9F8C70>> (for post_run_cell), with arguments args (<ExecutionResult object at 135ada34490, execution_count=4 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 135ada340d0, raw_cell="
parameters = {
    'learning_rate': {'values': [0.." store_history=True silent=False shell_futures=True cell_id=b896fac35c516b81> result=None>,),kwargs {}:


ConnectionAbortedError: [WinError 10053] Nawiązane połączenie zostało przerwane przez oprogramowanie zainstalowane w komputerze-hoście